# Assessment 2: Sqlite / Pandas Compare

Your business partner is interested in using either pandas as sqlite to calculate some statistics on this dataset.  

Set up a sqlite database to do a few comparisons with pandas.  Your business partner is interested in both run time and ease of use. 
    
Don't get too deep - let's just find out which is faster and which is easier to code up.

Do this analysis on the uncorrupted component of the data only.


Compare sqlite vs pandas on the following tasks:

	a) calculate top 25 most common 'makes'
    b) calculate most common 'Color' for each 'Make' 
    c) find the first ticket issued for each 'Make'


Choose either pandas or sqlite to answer the following question:

"Is an out-of-state license plate more likely to be expired at the time of recieving a ticket than an in-state license plate?"


In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [28]:
conn = sqlite3.connect('tickets.db')

## doing the analysis in pandas 

In [3]:
df = pd.read_csv('parking_citations.corrupted.csv', dtype=object)

In [4]:
df['Issue Date'] = pd.to_datetime(df['Issue Date'])

In [5]:
# get rid of the data without "make" info
df_make = df.loc[~df['Make'].isnull()]

### calculate top 25 most common makes

In [6]:
%timeit df_make['Make'].value_counts(normalize = True)[:25]

322 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
df_make['Make'].value_counts(normalize = True)[:25]

TOYT    0.165554
HOND    0.112899
FORD    0.087824
NISS    0.071445
CHEV    0.068175
BMW     0.045719
MERZ    0.040690
VOLK    0.034309
HYUN    0.030720
DODG    0.029320
LEXS    0.028573
KIA     0.023349
JEEP    0.023157
AUDI    0.019329
MAZD    0.018325
OTHR    0.016617
GMC     0.014318
CHRY    0.013154
INFI    0.013037
ACUR    0.012095
SUBA    0.010762
VOLV    0.009714
TOYO    0.009194
MITS    0.008684
CADI    0.007821
Name: Make, dtype: float64

### calculate most common color for each make, showing the top 5


In [8]:
%timeit (df_make.groupby(['Make', 'Color'])['Color'] .size().rename('top_count').reset_index().sort_values('top_count', ascending=False).groupby('Make').head(1))

516 ms ± 13.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
(df_make.groupby(['Make', 'Color'])['Color']
 .size()
 .rename('top_count')
 .reset_index()
 .sort_values('top_count', ascending=False)
 .groupby('Make')
 .head(1))[:5]

,Make,Color,top_count
4667,TOYT,GY,163238
1635,HOND,GY,116566
1255,FORD,WT,112313
648,CHEV,WT,78792
3312,NISS,GY,72622


### find the first ticket issued for each 'Make'

In [10]:
%timeit df_make.sort_values(by = ['Make','Issue Date']).groupby('Make').first().reset_index()[['Make', 'Issue Date']]

14.6 s ± 428 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
(df_make.sort_values(by = ['Make','Issue Date'])
 .groupby('Make')
 .first()
 .reset_index()
 [['Make', 'Issue Date']])[:10]

,Make,Issue Date
0,AAUD,2018-07-13
1,AAVH,2018-03-29
2,ABAR,2016-09-15
3,ABC,2016-05-10
4,ABRI,2017-04-18
5,ACC,2015-12-23
6,ACCR,2018-02-12
7,ACCU,2015-04-07
8,ACDI,2017-12-01
9,ACG,2016-03-15


In [12]:
# the above way is slow, trying this secondary way
%timeit df_make[['Make', 'Issue Date']].drop_duplicates(subset = 'Make', keep = 'first')
# this is much faster!! 

270 ms ± 8.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
df_make[['Make', 'Issue Date']].drop_duplicates(subset = 'Make', keep = 'first')[:10]

,Make,Issue Date
4,CHEV,2015-09-15
6,MAZD,2015-12-17
10,FORD,2015-12-22
11,CHRY,2015-12-22
14,TOYO,2015-12-27
19,HOND,2015-12-22
21,NISS,2015-12-18
29,TESL,2015-12-22
30,MBNZ,2015-12-22
31,ACUR,2015-12-21


### "Is an out-of-state license plate more likely to be expired at the time of recieving a ticket than an in-state license plate?"

In [14]:
# based on looking at the frequency counts of the license plate
# along with eye-balling the location field (googling the street addresses)
# one can deduce that what's considered "in-state" in this case is California
df_make['RP State Plate'].value_counts()[:10]

CA    4058883
AZ      38914
TX      27415
NV      26067
FL      19105
WA      17295
IL      12471
OR      12077
CO      11039
NY       8223
Name: RP State Plate, dtype: int64

In [15]:
# creating an "instate" field
df_make['instate'] = 0 
df_make.loc[df_make['RP State Plate'] == 'CA', 'instate'] = 1

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [16]:
# fields needed: issue date, plate expiry date, instate
# got rid of any without an expiration date
df_make = df_make.loc[~(df_make['Plate Expiry Date'].isnull())]

In [17]:
# only keep records with the right expiry dates
df_make = df_make.loc[df_make['Plate Expiry Date'].str.len() == 8]

In [18]:
# extract year and month 
df_make['year'] = df_make['Plate Expiry Date'].str[:4].astype('int')
df_make['month'] = df_make['Plate Expiry Date'].str[4:6].astype('int')
df_make['day'] = 1

In [19]:
df_make = df_make.loc[(df_make['year'].between(1990, 2020, inclusive=True))&(df_make['month'].between(1, 12, inclusive=True))]


In [20]:
df_make['plate_expiry_date'] = pd.to_datetime(df_make[['year', 'month', 'day']])

In [21]:
df_make['expired'] = 0
df_make.loc[(df_make['plate_expiry_date'] < df_make['Issue Date']), 'expired'] = 1

In [22]:
(pd.crosstab(df_make['expired'], df_make['instate'], 
             normalize = 'columns', margins = True))

instate,0,1,All
expired,,,
0,0.764624,0.780158,0.779226
1,0.235376,0.219842,0.220774


### Conclusion: a vehicle that's instate is less likely to be expired than an out of state plate

## doing the analysis in sqlite

In [23]:
# reload the untouched data
df_make = df.loc[~df['Make'].isnull()]

In [29]:
df_make.to_sql('tickets', con = conn)

### calculate top 25 most common makes

In [39]:
query = """
SELECT * FROM (
    SELECT Make, COUNT(*) 
    FROM tickets
    GROUP BY Make
    ORDER BY 2 desc
)
LIMIT 25
"""

In [40]:
%timeit df = pd.read_sql_query(query, conn)

2.94 s ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
df = pd.read_sql_query(query, conn)
df

,Make,COUNT(*)
0,TOYT,721411
1,HOND,491961
2,FORD,382695
3,NISS,311324
4,CHEV,297076
5,BMW,199221
6,MERZ,177307
7,VOLK,149501
8,HYUN,133864
9,DODG,127764


### calculate most common color for each make, showing the top 5


In [47]:
query = """
WITH test AS (
SELECT *
, ROW_NUMBER() OVER (PARTITION BY Make) AS row_num
FROM
    (
    SELECT Make
    , Color
    , COUNT(*) AS count
    FROM tickets
    GROUP BY Make, Color
    ORDER BY Make, 3 DESC
    )
)
SELECT *
FROM test
WHERE row_num = 1
ORDER BY count DESC
"""

In [43]:
%timeit df = pd.read_sql_query(query, conn)

5.54 s ± 125 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
df = pd.read_sql_query(query, conn)
df[:5]

,Make,Color,count,row_num
0,TOYT,GY,163238,1
1,HOND,GY,116566,1
2,FORD,WT,112313,1
3,CHEV,WT,78792,1
4,NISS,GY,72622,1


### find the first ticket issued for each 'Make'

In [36]:
query = """
SELECT Make
, "Issue Date"
FROM (
    SELECT Make
    , "Issue Date"
    , ROW_NUMBER() OVER (PARTITION BY Make) AS row_num
    FROM tickets
    GROUP BY Make, "Issue Date"
    ORDER BY Make, "Issue Date"
    )
WHERE row_num = 1
"""

In [37]:
%timeit df = pd.read_sql_query(query, conn)

5.88 s ± 73.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
df = pd.read_sql_query(query, conn)
df[:5]

,Make,Issue Date
0,AAUD,2018-07-13 00:00:00
1,AAVH,2018-03-29 00:00:00
2,ABAR,2016-09-15 00:00:00
3,ABC,2016-05-10 00:00:00
4,ABRI,2017-04-18 00:00:00
